# Using the simulator on Opentrons

In this notebook you will learn how to use the simulator to test out methods.

## Setting up a connection with the robot

As described in the [basic liquid handling tutorial](basic), we will use the {class}`~pylabrobot.liquid_handling.liquid_handler.LiquidHandler` class to control the robot. This time, however, instead of using the Hamilton {class}`~pylabrobot.liquid_handling.backends.hamilton.STAR.STAR` backend, we are using virtual the {class}`~pylabrobot.liquid_handling.backends.simulation.simulator_backend.SimulatorBackend` backend. This means that liquid handling will work exactly the same, but the commands are sent to the simulator instead of a real physical robot.

In [ ]:
%load_ext autoreload
%autoreload 2
# Test move, branch main to learn_PLR

: 

In [ ]:
from pylabrobot.liquid_handling import LiquidHandler
from pylabrobot.liquid_handling.backends.simulation.simulator_backend import SimulatorBackend

: 

In [ ]:
### QUESTION::
"""
  - For OT: HOW DO I FIND OUT WHAT PIPET IS ON THE HEAD? AND HOW DO I CHANGE THE PIPETS ON THE OT HEAD??
  - HOW CAN I DEFINE USING PIPET 1 AND PIPET 2? IF i have a robot with two of the same pipets on(aka our Tecan evo)
  - CAN I USE LLD ON HAMILTON ROBOT?
  - How can i override lld?
  - HOW CAN I PLACE LIDS ON PLATES? CAN I STACK PLATES ON TOP OF EACH OTHER?
  - HOW DO I REMOVE A LID WITH THE ISWAP ON THE HAMILTON
  - Difference between STAR and STARLet backend?? -> the length. STARLet is shorter.
  - How do i drop tips into trash???
  - Is there no tube runners for hamilton? ex: RGT_CAR_3R_A01, RGT_CAR_5R_A00, SMP_CAR_24_A00
  - How do i raise the tip 5 mm from the bottom of the plate? From where it pipets.
  - Does opentrons modules also work on PLR? (shaker, heater, PCR)
  - What is ChatterBox? in using-trackers.ipynb: lh = LiquidHandler(backend=ChatterBoxBackend(num_channels=8), deck=STARLetDeck())
  - How to copy the deck of Hamilton into PLR
"""

### TO DO FOR ME:
"""
  - ADD PRODUCT NUMBERS TO ALL TIP POSITIONS. USE THE SOFTWARE SO YOU CAN CROSS REFERENCE. The current structure assumes that the user has hamilton software to find out what the different carriers are.
"""

# from pylabrobot.liquid_handling.backends.opentrons_backend import OpentronsBackend

: 

In [ ]:
## For Hamilton:
# from pylabrobot.resources.hamilton import STARLetDeck

## For Opentrons:
# from pylabrobot.resources.opentrons import OTDeck # deck

from pylabrobot.resources.opentrons.deck import OTDeck

: 

In [ ]:
# ## For Hamilton:
# sb = SimulatorBackend(open_browser=False)
# lh = LiquidHandler(backend=sb, deck=STARLetDeck())

sb = SimulatorBackend(ws_host="127.0.0.1", fs_host="127.0.0.1")
# sb = SimulatorBackend(ws_host="0.0.0.0", fs_host="0.0.0.0")
lh = LiquidHandler(backend=sb, deck=OTDeck())

: 

Calling {func}`~pylabrobot.liquid_handling.liquid_handler.LiquidHandler.setup` will set up the simulation server and open it in a new browser tab.

In [ ]:
await lh.setup()

: 

For the optimal experience, we recommend that you run the notebook and simulator side by side.

In [ ]:
sb.wait_for_connection()

: 

![The empty simulator](./img/simulator/empty.png)

## Assigning plates and tips

With the simulator, {func}`assigning resources <pylabrobot.resources.Deck.assign_child_resource>` has the additional affect of placing the resources on the simulated deck. They will appear immediately.

In [ ]:
## For Hamilton:
# from pylabrobot.resources import (
#     TIP_CAR_480_A00,
#     PLT_CAR_L5AC_A00,
#     Cos_96_DW_1mL,
#     HTF_L
# )

## For Opentrons:
from pylabrobot.resources.opentrons import (
  opentrons_96_filtertiprack_20ul
)
from pylabrobot.resources.corning_costar import (
  Cos_96_EZWash,
  Cos_96_DW_1mL
)


: 

In [ ]:
# For Hamilton:
# tip_car = TIP_CAR_480_A00(name='tip carrier')
# tip_car[0] = tips = HTF_L(name='tips_01')
# tip_car[1] = HTF_L(name='tips_02')
# tip_car[2] = HTF_L(name='tips_03')
# tip_car[3] = HTF_L(name='tips_04')
# tip_car[4] = HTF_L(name='tips_05')

## For Opentrons:
tips1 = opentrons_96_filtertiprack_20ul(name="tip_rack1")
tips2 = opentrons_96_filtertiprack_20ul(name="tip_rack2")
plate1 = Cos_96_EZWash(name="plate1")
plate2 = Cos_96_DW_1mL(name="plate2")


: 

In [ ]:
## For Hamilton:
# lh.deck.assign_child_resource(tips rails=15)

## OPS OPS WE ARE USING asign_child_at_slot not RESOURCE!!!

## For Opentrons:
# lh.deck.assign_child_resource(tips1, location=3, reassign=False)

lh.deck.assign_child_at_slot(tips1, slot=3)
lh.deck.assign_child_at_slot(tips2, slot=5)
lh.deck.assign_child_at_slot(plate1, slot=11)
lh.deck.assign_child_at_slot(plate2, slot=10)


: 

In [ ]:
# summary function that displayes deck
print(lh.deck.summary())

: 

In [ ]:
# get slot position of resource
lh.deck.get_slot(tips1)

: 

![The simulator after the resources have been assigned](./img/simulator/assignment.png)

## Build the deck layout: placing resources

Where you would manually place the resources like tips and liquid on the deck when using a physical system, with the Simulator you can add them using code.

### Tips

Let's use {func}`~pylabrobot.liquid_handling.backends.simulation.SimulatorBackend.fill_tip_rack` to place tips at all spots in the tip rack in location `0`.

In [ ]:
# same code only change is the resource name: tip_rack1
tiprack = lh.get_resource("tip_rack1")
tiprack


: 

In [ ]:
await sb.fill_tip_rack(tiprack)

: 


You can precisely control the presence of tips using {func}`~pylabrobot.liquid_handling.backends.simulation.SimulatorBackend.edit_tips`.

In [ ]:
tips4 = lh.get_resource("tip_rack2")
await sb.edit_tips(tips4, pattern=[[True]*6 + [False]*6]*8)

: 

In [ ]:
await sb.edit_tips(lh.get_resource("tip_rack2"), pattern=[[True, False]*6]*8)

: 

In [ ]:
await sb.edit_tips(lh.get_resource("tip_rack2"), pattern=[[True, True, False, False]*3]*8)

: 

### Liquids

Adding liquid to wells works similarly. You can use {func}`~pylabrobot.liquid_handling.backends.simulation.SimulatorBackend.adjust_well_volume` to adjust the volume of individual wells in each resource. Note that the opacity of the well matches the volume of the well.

In [ ]:
plate_1_liquids = [[(None, 500)]]*96
await sb.adjust_wells_liquids(plate1, liquids=plate_1_liquids)

: 

In [ ]:
plate_2_liquids = [[(None, 100)], [(None, 500)]]*(96//2)
await sb.adjust_wells_liquids(plate2, liquids=plate_2_liquids)

: 

Using the simulator backend we have adjusted the volume in the simulator, which you can best compare to adding liquid in reality. Now we need to update the wells (that live in Python) to reflect how much volume is in them, so that LiquidHandler can validate your actions. This is done using {func}`~pylabrobot.resources.Plate.set_well_liquids`. Note that this can be done in all liquid handling protocols, not just the simulator.

In [ ]:
plate1.set_well_liquids(plate_1_liquids)
plate2.set_well_liquids(plate_2_liquids)

: 

![Simulator after the tips have been placed and the volumes adjusted](./img/simulator/resources.png)

## Liquid handling

Once the layout is complete, you can run the same commands as described in the [basic liquid handling tutorial](basic).

### Picking up tips

In [ ]:
tip_0 = lh.get_resource("tip_rack1")

: 

In [ ]:
# await lh.pick_up_tips(tip_0["A1", "B2", "C3", "D4"])
await lh.pick_up_tips(tip_0["A1"])


: 

In [22]:
# await lh.drop_tips(tip_0["A1"])
## TRY TO THROUGH IN TRASH!!!???
# trash_container
from pylabrobot.resources.trash import Trash

trash = lh.deck.get_trash_area()
# n must match with the number of pipets
n=1

offsets = trash.get_2d_center_offsets(n=n)

await lh.drop_tips(tip_spots=[trash]*n, offsets=offsets)

### Aspirating and dispensing

In [23]:
await lh.pick_up_tips(tip_0["A1"])

In [24]:
plate = lh.get_resource("plate1")

In [25]:
## OPS OPS SOME ERROR!? REASON== robot head is set to use 20 or 10 ul pipet. How do i change that??
# await lh.aspirate(plate["A2"], vols=[300])
await lh.aspirate(plate["A2"], vols=[5])

## FOUND BUG! IF LIQUID IS HIGHER THAN PIPET VOLUME OR TIP VOLUME THEN SIMULATION WILL GO INTO AN INFINITE LOOP

In [26]:
await lh.dispense(plate2["A1"], vols=[5])

In [27]:
await lh.drop_tips(tip_0["A1"])

### Aspirating using CoRe 96

The CoRe 96 head supports liquid handling operations for 96 channels at once. Here's how to use:

- {func}`~pylabrobot.liquid_handling.liquid_handler.LiquidHandler.pick_up_tips96` for picking up 96 tips;
- {func}`~pylabrobot.liquid_handling.liquid_handler.LiquidHandler.aspirate_plate` for aspirating liquid from an entire plate at once;
- {func}`~pylabrobot.liquid_handling.liquid_handler.LiquidHandler.dispense_plate` for dispensing liquid to an entire plate at once;
- {func}`~pylabrobot.liquid_handling.liquid_handler.LiquidHandler.drop_tips96` for dropping tips to the tip rack.


In [27]:
### INSTEAD O CORE 96 IMPLEMENT USING PIPET NUMER 2!

await lh.pick_up_tips96(tiprack)

In [28]:
await lh.aspirate_plate(plt_car[0].resource, volume=200)

In [29]:
await lh.dispense_plate(plt_car[2].resource, volume=200)

In [30]:
await lh.drop_tips96(tiprack)

![The simulator after the liquid handling operations completed](./img/simulator/after_lh.png)

## Shutting down

When you're done, remember to shut down the simulator by calling {func}`~pylabrobot.liquid_handling.liquid_handler.LiquidHandler.stop`.

In [31]:
await lh.stop()

connection handler failed
Traceback (most recent call last):
  File "c:\Users\vikmol\OneDrive\Dokumenter\GitHub\pylabrobot_DALSA\venv\lib\site-packages\websockets\legacy\server.py", line 236, in handler
    await self.ws_handler(self)
  File "c:\users\vikmol\onedrive\dokumenter\github\pylabrobot_dalsa\pylabrobot\liquid_handling\backends\websocket.py", line 138, in _socket_handler
    await self.handle_event(data.get("event"), data)
  File "c:\users\vikmol\onedrive\dokumenter\github\pylabrobot_dalsa\pylabrobot\liquid_handling\backends\websocket.py", line 112, in handle_event
    await self.websocket.send(json.dumps({"event": "pong"}))
  File "c:\users\vikmol\onedrive\dokumenter\github\pylabrobot_dalsa\pylabrobot\liquid_handling\backends\websocket.py", line 69, in websocket
    raise RuntimeError("No websocket connection has been established.")
RuntimeError: No websocket connection has been established.


# Bug test Does trashbin work in Simulation

In [1]:
from pylabrobot.liquid_handling import LiquidHandler
from pylabrobot.liquid_handling.backends.simulation.simulator_backend import SimulatorBackend
from pylabrobot.resources.opentrons.deck import OTDeck

sb = SimulatorBackend(ws_host="127.0.0.1", fs_host="127.0.0.1")
# lh = LiquidHandler(backend=sb, deck=OTDeck())
lh = LiquidHandler(backend=sb, deck=OTDeck(no_trash=True))

# lh = LiquidHandler(backend=OpentronsBackend(host="10.199.253.164"), deck=OTDeck(no_trash=True)) # This seems to work

await lh.setup()

Websocket server started at http://127.0.0.1:2121
File server started at http://127.0.0.1:1337 . Open this URL in your browser.


In [2]:
sb.wait_for_connection()

In [3]:
from pylabrobot.resources.opentrons import (
  opentrons_96_filtertiprack_20ul
)
from pylabrobot.resources.corning_costar import (
  Cos_96_EZWash,
  Cos_96_DW_1mL
)

tips1 = opentrons_96_filtertiprack_20ul(name="tip_rack1")
tips2 = opentrons_96_filtertiprack_20ul(name="tip_rack2")
plate1 = Cos_96_EZWash(name="plate_01")
plate2 = Cos_96_DW_1mL(name="plate_02")

lh.deck.assign_child_at_slot(tips1, slot=3)
lh.deck.assign_child_at_slot(tips2, slot=5)
lh.deck.assign_child_at_slot(plate1, slot=11)
lh.deck.assign_child_at_slot(plate2, slot=10)

print(lh.deck.summary())


Deck: 624.3mm x 565.2mm

+-----------------+-----------------+-----------------+
|                 |                 |                 |
| 10: plate_02    | 11: plate_01    | 12: Empty       |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|  7: Empty       |  8: Empty       |  9: Empty       |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|  4: Empty       |  5: tip_rack2   |  6: Empty       |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|  1: Empty       |  2: Empty       |  3: tip_rack1   |
|                 |                 |                 |
+-----------------+-----------------+-----------------+



In [4]:
## SImulating specific code
tiprack = lh.get_resource("tip_rack1")
tiprack
await sb.fill_tip_rack(tiprack)

In [5]:
tiprack = lh.get_resource("tip_rack1")
await lh.pick_up_tips(tiprack["A1"])

In [6]:
from pylabrobot.resources.trash import Trash

In [7]:
lh.deck._assign_trash()

# lh.deck._assign_trash()

In [8]:
trash = lh.deck.get_trash_area()

In [9]:
print(lh.deck.summary())


Deck: 624.3mm x 565.2mm

+-----------------+-----------------+-----------------+
|                 |                 |                 |
| 10: plate_02    | 11: plate_01    | 12: trash_co... |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|  7: Empty       |  8: Empty       |  9: Empty       |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|  4: Empty       |  5: tip_rack2   |  6: Empty       |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|  1: Empty       |  2: Empty       |  3: tip_rack1   |
|                 |                 |                 |
+-----------------+-----------------+-----------------+



In [12]:
print(lh.sb.defined_labware)

AttributeError: 'LiquidHandler' object has no attribute 'sb'

In [13]:
await lh.drop_tips(tip_spots=[trash]*1)

NoTipError: Channel 0 does not have a tip.